# HyperSpy Fitting tutorial

This tutorial shows the basics of model (currently only 1D) fitting in HyperSpy from the grounds up.

__Minimum required version:__ HyperSpy 1.6

## Authors:

- 13/04/2015 Tomas Ostasevicius - Developed for HyperSpy workshop at University of Cambridge
- 01/06/2016 Tomas Ostasevicius - updated and expanded for HyperSpy workshop at Scandem conference 2016
- 22/07/2016 Tomas Ostasevicius - updated for HyperSpy version 1.0
- 19/04/2021 Francisco de la Peña - new synthetic dataset. Add standard deviation discussion.
- 20/04/2021 Francisco de la Peña - Tweak dataset to always fail with `iterpath="flyback"`. Add bounds, fixing parameters and standard deviation check.

<a id='terms'></a>
## Terminology and relationships
<a href='#top'>[back to top]</a> 

In order to use fitting in HyperSpy more effectively, it is useful to understand our structure for curve fitting.

There are three main things, related to fitting:

__1. Model__
can be thought of as a simple box (cooking pot), where we have to put our ingredients. Without anything inside, it is not of much use in this case. Once we add some things to it and mix it a bit (do the actual fitting), however, we have our complete dish!

__2. Component__ is the main building block (ingredient) of our model. Here we mix and match what components we need (or want) for the particular case of signal. 

Examples: 
- Lorentzian (Cauchy)
- Gaussian
- Voigt (a combination of Lorentzian and Gaussian)
- Offset (i.e. constant background)
- Exponential function
- ...
- [create your own or use the very specialised ones!]

Each of the components is ultimately just a function that has variables that change the (shape of the) output. Such a variable in HyperSpy is called a __parameter__. The model is built by combining *linearly* the components.


__3. Parameter__ is the knob that the fitting routine adjusts for a good fit. Each component must include at least one parameter in order to be able to change when fitting. A parameter is also the object that we may limit or have to adjust when the result of the fit is not satisfactory. 

Ultimately, a parameter is the only important thing, as far as the fitting is concerned - components are just smart and convenient boxes to combine parameters into functions, and a model is just a box for a collection of components.

For now, let's just keep the rough structure in our heads and look at other things!



<a id='top'></a>
## Importing and exploring the relevant docstrings

HyperSpy, like all other Python libraries, first has to be imported in your Python setup in order to be used.
Once it is, all the relevant commands can be looped up using the autocompletion feature of the IPython. 

Lets import the HyperSpy and set up plotting.

In [ ]:
%matplotlib notebook
import hyperspy.api as hs

Once imported, all the HyperSpy commands are available via the 
> `hs.<something>`

interface. You can also look for the help with any Python object like this
> `help(<something>)`

In [ ]:
help(hs)

In [ ]:
help(hs.model.components1D)

<a id='loading'></a>
## Loading the data and creating the model
<a href='#top'>[back to top]</a> 

First you should have a spectrum (a particular kind of the `Signal` subclass!) you want to fit. Let's load a synthetic dataset with some curves named 
> `"two_peaks.hspy"`

and have a look at it.

If you can't load the dataset, it means you most likely have not generated it yet. Please run the two cells <a href='#two_peaks'>at the end</a> of the notebook to do so.

In [ ]:
s = hs.load("two_peaks.hspy")

In [ ]:
s

In [ ]:
s.plot()

Exploring the dataset we notice that it consists of two peaks that at different points change position and height. In oder to measure the position, height and width of the two peaks at all position we will create a ``Model`` that consists of 2 gaussian functions.

<a id='creating_model'></a>
<a href='#top'>[back to top]</a> 

Creating a model now is simple - just pass the spectrum to the function 
> `model_reference = signal_reference.create_model()`

Let's reference the model by "`m`".

In [ ]:
m = s.create_model()

Let's look what's inside:

In [ ]:
m.components

As we can see, the model is still empty. That will not always be the case - for some types of signals, an automatic background component is added when creating a model, hence it's always good to check.

We can plot the model in exactly the same way as the signal:

The only difference from the model plot is that each data point is displayed individually. 

<a id='creating_components'></a>
<a href='#top'>[back to top]</a> 

To do anything with the model, we should __create__ some __components__ and add them. Let's create two gaussians, referenced as "g1" and "g2":

-----------------
P.S.: keep in mind that creating a component is a function - hence there should be brackets at the end! Such as 
> `our_component_reference = hs.model.components1D.example_component()`

In [ ]:
g1 = hs.model.components1D.GaussianHF()
g2 = hs.model.components1D.GaussianHF()

<a id='adding_comps'></a>
<a href='#top'>[back to top]</a> 

... and __add the components to__ our __model__. For that there are generally two ways:

Individually
> `our_model_reference.append(our_component_reference)`

or in lists (i.e. grouped by square brackets)
> `our_model_reference.extend([first_component_reference, second_component_reference])`

In [ ]:
m.extend([g1, g2])

Let's check how the model looks now:

In [ ]:
m.components

<a id='renaming_components'></a>
<a href='#top'>[back to top]</a> 

For our convenience we can __rename__ the __components__ as we choose, for example "wide" and "narrow" (note that the "g1" and "g2" are only references we created for them, not names of the components)

In [ ]:
g1.name = "wide"
g2.name = "narrow"

We can look at the model again to see the result

In [ ]:
m.components

<a id='checking_values'></a>
<a href='#top'>[back to top]</a>

To finally see the full structure (the one we looked at <a href='#terms'>here</a>), we can __print all of the parameter values__ of all components of the model.

In [ ]:
m.print_current_values()

To access the values, we have to look inside the components for the parameters. It can simply be done by following the pattern:
> `some_component_reference.parameter_name.value`

In this case the component references are the __g1__ and __g2__, while parameter names are __centre__, __A__ and __sigma__.

At this point it is good practice to store the model so that, if something goes wrong, we can go back to this stage:

In [ ]:
m.store(name="new")

In [ ]:
m.signal.models

Notice that there is another model stored in the signal, `ground truth`. We'll put it to use at end of this tutorial.

In [ ]:
g1.fwhm.value

Notice that we can access the component more conveniently using its name as follows:

In [ ]:
m.components.wide.fwhm.value

<a id='setting_values'></a>
<a href='#top'>[back to top]</a>

We can __set parameter values__ in exactly the same way. Let set `g1` `sigma` value to 30:

In [ ]:
m.components.wide.fwhm.value = 30

In [ ]:
m.print_current_values()

For convenience, we can also __set values "in bulk"__ for all components in the model. The required command is
> `m.set_parameters_value`

Set the area ("A" parameter) of both peaks to 500

In [ ]:
m.set_parameters_value('height', 100)

In [ ]:
m.print_current_values()

## Indexing the model and fitting a single spectrum

We will start by analyzing the data at one single pixel. For that we can using the same indexing syntax that we use for signals.

In [ ]:
m00 = m.inav[0, 0]

In [ ]:
m00.plot(plot_components=True)

In order to increase the chances of obtaining a good fit to the data, we must provide better starting parameters.

We could do that interactively, using ``m.gui()`` or manually as follows:

In [ ]:
m00.components.narrow.centre.value = 60
m00.components.wide.centre.value = 50

We are now ready to fit

In [ ]:
m00.fit()

The fit (notice that the model got updated in the figure) looks good!

Notice that the value of the parameters have changed to their optimal value. Also notice that the standard deviation has been estimated too.

In [ ]:
m00.print_current_values()

But how to know how good is the fit? 


Once the fit was performed, chi-squared ($\chi^2$), degrees of freedom and reduced chi-squared ($\chi^2_\nu$) of the fit are automatically calculated.

They are accessible with, respectively:
> `m2.chisq`

> `m2.dof`

> `m2.red_chisq`

Let's have a look at reduced $\chi^2$ by plotting it

In [ ]:
m00.red_chisq.data

That's far too big! (A good fit should have $a\chi^2_\nu$ of around 1.)

The issue is that we haven't defined the variance of the noise:

In [ ]:
m00.signal.metadata

For that we can use the following ``Signal`` method:

In [ ]:
m00.signal.estimate_poissonian_noise_variance()

Let's now fit. Because the noise variance is available, HyperSpy will now perform weighted non-linear least squares automatically.

In [ ]:
m00.fit()

In [ ]:
m00.red_chisq.data

That's much better!

In [ ]:
m00.print_current_values()

Notice that the value of the parameters and their standard deviation has changed. The previous values were biased because we haven't defined the noise variance. These values should be more accurate.

We can obtain an even more accurate result by using the current model to better estimate the noise variance:

In [ ]:
m00.signal.estimate_poissonian_noise_variance(expected_value=m00.as_signal())
m00.fit()
m00.print_current_values()

In [ ]:
m00.red_chisq.data

## Fitting the first line

Let's now analyze the first line before attempting to fit the whole dataset

In [ ]:
ml = m.inav[:, 0]

In [ ]:
ml.plot()

Notice that the parameters of the first pixels that we have estimated are already set.

Like before, we must estimate the noise variance

In [ ]:
ml.signal.estimate_poissonian_noise_variance()

In order to fit the whole line we must use the ``multifit()`` method

In [ ]:
ml.multifit()

In [ ]:
ml.red_chisq.plot()

That looks pretty good!

Let's have a look at the parameters

In [ ]:
ml.components.narrow.height.plot()

Interesting, the height seems to vary between 30 and 120.

In [ ]:
ml.components.narrow.centre.plot()

And the position varies sinusoidally between 40 and 60

In [ ]:
ml.components.narrow.fwhm.plot()

There doesn't seem to be any pattern in the variation of the FWHM, it seems to vary randomly around 6. Let's check if its standard variation to value ratio:

In [ ]:
ml.components.narrow.fwhm.as_signal().data.std() / ml.components.narrow.fwhm.as_signal().data.mean()

That's around 5%, which, given the noisiness of the data, is consistent with this  parameter not varying at all,

In [ ]:
ml.components.wide.fwhm.as_signal().data.std() / ml.components.wide.fwhm.as_signal().data.mean()

The wide peaks std to value ratio is just .6%, therefore it is reasonable to think that this parameter is fixed too.

More on this later.

## Fitting the whole model

Before fitting, it is a good idea storing the current state of the model. In this way, if we do something wrong, we can always return to this state

In [ ]:
s.models

In [ ]:
m.store("first line fitted")

In [ ]:
m.signal.models

In [ ]:
m.signal.estimate_poissonian_noise_variance()

In [ ]:
m.multifit()

All of those warnings suggest that something has indeed gone wrong.

In [ ]:
m.red_chisq.get_histogram().plot()

The large number of pixels with a high $\chi_{\nu}^2$ indicates that something is wrong with the fit

In [ ]:
m.red_chisq.plot()

It looks like something went wrong when jumping from the first line to the second, probably because the starting parameters copied from the last pixel of the first row do are not suitable for the first pixel of the second row. Let's restore the model and try fitting with a differing starting parameters strategy:

In [ ]:
m = m.signal.models.restore("first line fitted")

In [ ]:
m.multifit(iterpath="serpentine")

In [ ]:
m.red_chisq.get_histogram().plot()

In [ ]:
m.red_chisq.plot()

The fact that there is no contrast shows that this time the fit is good at all pixels in the dataset! This is because with `iterpath="serpentine"` the fitting routine advances one row at the end of each row without retourning to the beginning of the line.

Let's visualize the results:

In [ ]:
m.plot_results()

## Bounds and fixed components

Notice the the FWHM of the narrow component sometimes gets negative. It doesn't actually matter, but it makes plotting and analysis more challenging. To fix this issue we can constrain the FWHM of the gaussian:

In [ ]:
m = m.signal.models.restore("first line fitted")
m.components.narrow.fwhm.bmin = 5.5
m.components.narrow.fwhm.bmax = 6.5

In [ ]:
m.multifit(bounded=True, iterpath="serpentine")

In [ ]:
m.red_chisq.plot()

In [ ]:
m.plot_results()

It worked: there is no sign reversal in the narrow FWHM plot anymore.

Let's have a look at the histogram of the FWHM parameter of both peaks:

In [ ]:
m.components.narrow.fwhm.as_signal().get_histogram().plot()

In [ ]:
m.components.wide.fwhm.as_signal().get_histogram().plot()

The peaky shape of the histogram suggests that the parameter variation is only due to noise. The asymmetry is typical of the bias induced by approximating the Poisson noise with a gaussian (i.e. using weighted least-squares instead of an unbiased estimator such as maximum likelihood).

Let's fit again using maximum likelihood:

In [ ]:
m.multifit(optimizer="Nelder-Mead", loss_function="ML-poisson")

In [ ]:
m.components.narrow.fwhm.as_signal().get_histogram().plot()

In [ ]:
m.components.wide.fwhm.as_signal().get_histogram().plot()

Now the peaks are more symmetric, confirming that the biased loss function was the issue. Let's check their mean:

In [ ]:
m.components.wide.fwhm.as_signal().data.mean()

In [ ]:
m.components.narrow.fwhm.as_signal().data.mean()

This is very close to the actual values that in this case are 60 and 6. With this insight we can refit the model setting the 

This suggests that it would have been better to fix the FWHM of the peaks, let's try:

In [ ]:
m.store("fitted with ML-poisson")

In [ ]:
m = m.signal.models.restore("new")

In [ ]:
m.components.narrow.fwhm.value = 6
m.components.narrow.fwhm.free = False
m.components.wide.fwhm.value = 60
m.components.wide.fwhm.free = False

In [ ]:
m.plot()

In [ ]:
m.signal.axes_manager.indices = (0, 0)

In [ ]:
m.components.narrow.centre.value = 60
m.components.wide.centre.value = 50
m.set_parameters_value("height", 100)

In [ ]:
m.fit()

In [ ]:
m.multifit(iterpath="serpentine")

In [ ]:
m.red_chisq.get_histogram().plot()

## Standard deviation

We can also insptect the standard deviation maps:

In [ ]:
narrow_centre_value = m.components.narrow.centre.as_signal()
narrow_centre_std = m.components.narrow.centre.as_signal(field="std")

In [ ]:
hs.plot.plot_images([narrow_centre_value, narrow_centre_std], label=["centre", "centre std"], axes_decor="off")

Notice how the standard deviation image has a contrast? Is that expected? If yes, what should it be correlated with?

Let's now verify that the standard deviation estimation is correct. We know that, if correct, 66% percent of the residuals must fall between a 1 standard deviation interval. Let's use this property to verify the result:

The ground truth is actually stored in the signal:


In [ ]:
m.signal.models

In [ ]:
narrow_centre_gt = m.signal.models.restore("ground truth").components.narrow.centre.as_signal()

In [ ]:
residuals = (narrow_centre_gt - narrow_centre_value).data

In [ ]:
residuals_in_1sigma = len(residuals[(residuals > -narrow_centre_std.data) & (residuals < narrow_centre_std.data)])

In [ ]:
import numpy as np
100 * residuals_in_1sigma / np.prod(residuals.shape)

That is very close to 66% percent, so it works!

## Appendix I: User define components interesting components

Lets say we have a slightly stranger signal that we want to fit, like this one:

In [ ]:
s = hs.load('wobbly_peak.hspy')

In [ ]:
s.plot()

It's (as the name implies) composed of a sinus + gaussian + 2nd degree polynomial. However we don't have a `sin` component in the in-build library, so we'll just write our own:

In [ ]:
sin = hs.model.components1D.Expression('A*sin(b*x + c)',
                                        name='sin',)

Then just create and add all the additional components we might need: a gaussian and a polynomial

In [ ]:
m = s.create_model()
gaus = hs.model.components1D.Gaussian()
poly = hs.model.components1D.Polynomial(2)
m.extend([sin, gaus, poly])

In [ ]:
m.print_current_values()

The initial values do not seem to be very useful, so let's just plot the model, turn on the widgets, and we'll play until things seem close enough:

In [ ]:
m.plot()

m.gui()

And then fit it and look at the results!

In [ ]:
m.fit()

In [ ]:
m.print_current_values()

## Appendix II: Generating the synthetic data

<a id='two_peaks'></a>
### Two peaks
<a href='#top'>[back to top]</a> 


In [ ]:
import numpy as np
import hyperspy.api as hs

domain = 32 #size of the square domain
hfactor = 600
cent = (domain//2, domain//2)
y,x = np.ogrid[-cent[0]:domain-cent[0], -cent[1]:domain-cent[1]]

def gaussian2d(x, y, A=1, x0=0, y0=0, sigmax=20, sigmay=10):
    return A * np.exp(-((x-x0)**2 / 2 / sigmax ** 2 + (y-y0)**2 / 2 / sigmay ** 2))

center_narrow = 50 + 10 * np.sin(3 * np.pi * x / domain) * np.cos(4 * np.pi * y / domain)
center_wide = 50 + 10 * (-0.1 * np.sin(3 * np.pi * x / domain) * np.cos(4 * np.pi * y / domain))

r = np.sqrt(x**2 + y**2)
h_narrow = .5 * (.5 + np.sin(r)**2) * gaussian2d(x, y) *  hfactor
h_wide = (.5 + np.cos(r)**2) * gaussian2d(x, y) *  hfactor

s = hs.signals.Signal1D(np.ones((domain,domain, 1024)))
s.metadata.General.title = 'Two gaussians'
s.axes_manager[0].name = "x"
s.axes_manager[0].units = "nm"
s.axes_manager[1].name = "y"
s.axes_manager[1].units = "nm"

s.axes_manager[2].name = "Energy"
s.axes_manager[2].name = "eV"
s.axes_manager[2].scale = 0.1
m0 = s.create_model()

gs01 = hs.model.components1D.GaussianHF()
gs01.name = "wide"
m0.append(gs01)
gs01.fwhm.value = 60
gs01.centre.map['values'][:] = center_wide
gs01.centre.map['is_set'][:] = True
gs01.height.map['values'][:] = h_wide
gs01.height.map['is_set'][:] = True

gs02 = hs.model.components1D.GaussianHF()
gs02.name = "narrow"
m0.append(gs02)
gs02.fwhm.value = 6
gs02.centre.map['values'][:] = center_narrow
gs02.centre.map['is_set'][:] = True
gs02.height.map['values'][:] = h_narrow
gs02.height.map['is_set'][:] = True
s.data = m0.as_signal().data
s.add_poissonian_noise(random_state=0)
m0.store("ground truth")
s.save("two_peaks.hspy", overwrite=True)

<a id='wobbly_peak'></a>
### Wobbly peak
<a href='#top'>[back to top]</a> 


In [ ]:
import numpy as np
import hyperspy.api as hs

k = 1
alpha = 15
amp = 3

gaus_position = 15
gaus_width = 5
gaus_A = 300

gradient = 0.6
offset= 3

sin_component = hs.model.components1D.Expression('A * sin(k*x + alpha)', name='sin', k=k,
                                               alpha=alpha, A=amp)
gaus = hs.model.components1D.Gaussian(A=gaus_A, sigma=gaus_width, centre=gaus_position)

poly = hs.model.components1D.Polynomial(1)

poly.coefficients.value = (gradient, offset)

axis = np.linspace(0, 30, 3000, dtype='double')

result = sin_component.function(axis)+ gaus.function(axis) + poly.function(axis)
s = hs.signals.Signal1D(result)
s.axes_manager[0].name = 'x'
s.axes_manager[0].scale = 0.1
s.axes_manager[0].offset = 0

s.metadata.General.author = 'Tomas Ostasevicius'
s.metadata.General.title = 'Sin + poly(2) + Gaussian'
s.save('wobbly_peak', overwrite=True)